<a href="https://colab.research.google.com/github/hyunjin2123/BA_ToDo/blob/main/gs25%2C%EB%82%A0%EC%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#한글 폰트

%matplotlib inline
import matplotlib as mpl               # 기본 설정
import seaborn as sns           # Seaborn 로딩하기
import matplotlib.pyplot as plt        # 그래프 관련
import matplotlib.font_manager as fm   # 폰트 관련
import pandas as pd
import numpy as np
import lightgbm                         #LGBM 활용 - 기존 tree 기반 알고리즘과 달리 tree구조가 수직적으로 확장 *GBM이란 틀린부분에 가중치를 더하면서 진행하는 알고리즘
from tqdm import tqdm
import warnings

mpl.rcParams['axes.unicode_minus'] = False  ## 마이너스 부호가 깨지는 것 방지

#한글 사용하는 방법 !! 
#1. 먼저 나눔 폰트 설치
#2. 전체 폰트 갯수 중  나눔 폰트 갯수가 뜨는지 확인
#3. 런타임-런타임 다시 시작 후, 차트 plot 다시 실행

!apt-get update -qq
!apt-get install fonts-nanum* -qq              

sys_font=fm.findSystemFonts()
print(f"sys_font number: {len(sys_font)}")
print(sys_font)                               

nanum_font = [f for f in sys_font if 'Nanum' in f]
print(f"nanum_font number: {len(nanum_font)}") 
nanum_font

In [56]:
# 설치된 나눔글꼴중 원하는 폰트의 전체 경로 가져오기 
path = '/usr/share/fonts/truetype/nanum/NanumSquareB.ttf'  
font_name = fm.FontProperties(fname=path, size=14).get_name()
print(font_name)

plt.rc('font', family=font_name)

NanumSquare


In [ ]:
# 간단하게 그림을 그릴 수 있는 mglearn 라이브러리 사용 (!pip install mglearn 명령어로 설치)
!pip install mglearn
import mglearn

In [58]:
# 경고 끄기
pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings(action='ignore')

In [77]:
#GS25 상품 판매 데이터 불러오기

from pandas.core.frame import DataFrame

import pandas as pd

# csv 데이터 가져오기
df_GS25 = pd.read_csv("/content/drive/My Drive/Colab Notebooks/BigMining/최종과제/GS25.csv", encoding="cp949")

print(df_GS25.columns)
print(df_GS25.info())      #데이터정보파악 행, 열, 결측치 등
display(df_GS25.head(5))    #처음 5행 확인

Index(['korea_cvs.pvn_nm', 'korea_cvs.sale_dt', 'korea_cvs.gen_cd',
       'korea_cvs.age_cd', 'korea_cvs.category', 'korea_cvs.adj_qty',
       'korea_cvs.bor_nm'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2707786 entries, 0 to 2707785
Data columns (total 7 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   korea_cvs.pvn_nm    object
 1   korea_cvs.sale_dt   int64 
 2   korea_cvs.gen_cd    object
 3   korea_cvs.age_cd    object
 4   korea_cvs.category  object
 5   korea_cvs.adj_qty   int64 
 6   korea_cvs.bor_nm    object
dtypes: int64(2), object(5)
memory usage: 144.6+ MB
None


,korea_cvs.pvn_nm,korea_cvs.sale_dt,korea_cvs.gen_cd,korea_cvs.age_cd,korea_cvs.category,korea_cvs.adj_qty,korea_cvs.bor_nm
0,서울특별시,20160101,F,00~19,라면,7,종로구
1,서울특별시,20160101,F,20~39,과자,655,종로구
2,서울특별시,20160101,F,20~39,라면,282,종로구
3,서울특별시,20160101,F,20~39,마스크,35,종로구
4,서울특별시,20160101,F,20~39,맥주,183,종로구


In [78]:
df_GS25.columns=['광역시','ymd','성별','연령대','상품','수량','시군구']
df_GS25

,광역시,ymd,성별,연령대,상품,수량,시군구
0,서울특별시,20160101,F,00~19,라면,7,종로구
1,서울특별시,20160101,F,20~39,과자,655,종로구
2,서울특별시,20160101,F,20~39,라면,282,종로구
3,서울특별시,20160101,F,20~39,마스크,35,종로구
4,서울특별시,20160101,F,20~39,맥주,183,종로구
...,...,...,...,...,...,...,...
2707781,경기도,20181230,M,40~59,과자,7,여주시이천시
2707782,경기도,20181231,F,00~19,맥주,7,여주시이천시
2707783,경기도,20181231,F,20~39,생리대,7,여주시이천시
2707784,경기도,20181231,F,60~99,생리대,7,여주시이천시


In [134]:
#종로구 자료만 선택
df_GS25_final = df_GS25[df_GS25['시군구'] == '종로구'] # 시군구 열이 '종로구'인 행들만 추출
df_GS25_final

,광역시,ymd,성별,연령대,상품,수량,시군구
0,서울특별시,20160101,F,00~19,라면,7,종로구
1,서울특별시,20160101,F,20~39,과자,655,종로구
2,서울특별시,20160101,F,20~39,라면,282,종로구
3,서울특별시,20160101,F,20~39,마스크,35,종로구
4,서울특별시,20160101,F,20~39,맥주,183,종로구
...,...,...,...,...,...,...,...
2519452,서울특별시,20181231,M,00~19,면도기,7,종로구
2519453,서울특별시,20181231,M,20~39,생리대,7,종로구
2519454,서울특별시,20181231,M,20~39,스타킹,7,종로구
2519455,서울특별시,20181231,M,40~59,마스크,7,종로구


In [135]:
df_GS25_final = df_GS25_final.drop(['광역시','성별','연령대','시군구'], axis=1)
df_GS25_final

,ymd,상품,수량
0,20160101,라면,7
1,20160101,과자,655
2,20160101,라면,282
3,20160101,마스크,35
4,20160101,맥주,183
...,...,...,...
2519452,20181231,면도기,7
2519453,20181231,생리대,7
2519454,20181231,스타킹,7
2519455,20181231,마스크,7


In [136]:
df_GS25_final.set_index('ymd')
df_GS25_final

,ymd,상품,수량
0,20160101,라면,7
1,20160101,과자,655
2,20160101,라면,282
3,20160101,마스크,35
4,20160101,맥주,183
...,...,...,...
2519452,20181231,면도기,7
2519453,20181231,생리대,7
2519454,20181231,스타킹,7
2519455,20181231,마스크,7


In [137]:
# 상품을 변수로 변경(집계 포함)
상품_val = df_GS25_final["상품"].unique()
상품_list =  상품_val.tolist()
상품_list

['라면',
 '과자',
 '마스크',
 '맥주',
 '생리대',
 '생수',
 '숙취해소제',
 '스타킹',
 '아이스크림',
 '탄산음료',
 '면도기',
 '우산']

In [140]:
GS25_df = pd.DataFrame(df_GS25_final,  columns = 상품_list).sum()   #데이터프레임 형태로 각 상품 일자별 판매수량 확인하는 방법 찾아야 함.
display(GS25_df.head())


라면     0.0
과자     0.0
마스크    0.0
맥주     0.0
생리대    0.0
dtype: float64

In [23]:
#기상청 기상 데이터 불러오기

# csv 데이터 가져오기
df_wether = pd.read_csv("/content/drive/My Drive/Colab Notebooks/BigMining/최종과제/기상데이터.csv", encoding="cp949")

print(df_wether.columns)
print(df_wether.info())      #데이터정보파악 행, 열, 결측치 등
display(df_wether.head(5))    #처음 5행 확인

Index(['bigcon_weather.tm', 'bigcon_weather.stn_id', 'bigcon_weather.pvn_nm',
       'bigcon_weather.bor_nm', 'bigcon_weather.max_ta',
       'bigcon_weather.max_ws', 'bigcon_weather.min_ta',
       'bigcon_weather.avg_ta', 'bigcon_weather.avg_rhm',
       'bigcon_weather.avg_ws', 'bigcon_weather.sum_rn'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59113 entries, 0 to 59112
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   bigcon_weather.tm       59113 non-null  int64  
 1   bigcon_weather.stn_id   59113 non-null  int64  
 2   bigcon_weather.pvn_nm   59113 non-null  object 
 3   bigcon_weather.bor_nm   59113 non-null  object 
 4   bigcon_weather.max_ta   58826 non-null  float64
 5   bigcon_weather.max_ws   58778 non-null  float64
 6   bigcon_weather.min_ta   58826 non-null  float64
 7   bigcon_weather.avg_ta   58814 non-null  float64
 8   bigcon_weather.avg_rhm  43

,bigcon_weather.tm,bigcon_weather.stn_id,bigcon_weather.pvn_nm,bigcon_weather.bor_nm,bigcon_weather.max_ta,bigcon_weather.max_ws,bigcon_weather.min_ta,bigcon_weather.avg_ta,bigcon_weather.avg_rhm,bigcon_weather.avg_ws,bigcon_weather.sum_rn
0,20160101,98,경기도,동두천시,6.5,3.8,-6.4,-0.2,74.0,0.9,0.0
1,20160102,98,경기도,동두천시,10.1,3.2,1.9,5.4,70.0,1.3,0.0
2,20160103,98,경기도,동두천시,9.0,2.4,2.1,4.7,82.0,0.7,0.0
3,20160104,98,경기도,동두천시,8.0,5.3,-3.0,1.9,55.0,2.2,0.0
4,20160105,98,경기도,동두천시,3.1,5.6,-6.4,-2.8,36.0,1.8,0.0


In [34]:
df_wether_temp = df_wether.drop(columns = ['bigcon_weather.stn_id','bigcon_weather.max_ta','bigcon_weather.max_ws','bigcon_weather.min_ta'])
df_wether_temp.head()

,bigcon_weather.tm,bigcon_weather.pvn_nm,bigcon_weather.bor_nm,bigcon_weather.avg_ta,bigcon_weather.avg_rhm,bigcon_weather.avg_ws,bigcon_weather.sum_rn
0,20160101,경기도,동두천시,-0.2,74.0,0.9,0.0
1,20160102,경기도,동두천시,5.4,70.0,1.3,0.0
2,20160103,경기도,동두천시,4.7,82.0,0.7,0.0
3,20160104,경기도,동두천시,1.9,55.0,2.2,0.0
4,20160105,경기도,동두천시,-2.8,36.0,1.8,0.0


In [35]:
df_wether_temp.columns=['ymd','광역시','시군구','평균기온','습도','평균풍속','강수량']
df_wether_temp

,ymd,광역시,시군구,평균기온,습도,평균풍속,강수량
0,20160101,경기도,동두천시,-0.2,74.0,0.9,0.0
1,20160102,경기도,동두천시,5.4,70.0,1.3,0.0
2,20160103,경기도,동두천시,4.7,82.0,0.7,0.0
3,20160104,경기도,동두천시,1.9,55.0,2.2,0.0
4,20160105,경기도,동두천시,-2.8,36.0,1.8,0.0
...,...,...,...,...,...,...,...
59108,20181227,경기도,연천군,-10.2,35.0,1.6,0.0
59109,20181228,경기도,연천군,-12.2,48.0,1.1,0.0
59110,20181229,경기도,연천군,-9.3,46.0,1.2,0.0
59111,20181230,경기도,연천군,-10.3,56.0,0.8,0.0


In [42]:
#기상데이터도 종로구 자료만 선택
df_wether_final = df_wether_temp[df_wether_temp['시군구'] == '종로구'].head() # 시군구 열이 '종로구'인 행들만 추출
df_wether_final

,ymd,광역시,시군구,평균기온,습도,평균풍속,강수량
2192,20160101,서울특별시,종로구,1.2,73.0,1.6,0.0
2193,20160102,서울특별시,종로구,5.7,77.0,2.0,0.0
2194,20160103,서울특별시,종로구,6.5,81.0,1.8,0.0
2195,20160104,서울특별시,종로구,2.0,54.0,3.1,0.0
2196,20160105,서울특별시,종로구,-2.7,39.0,2.3,0.0


In [143]:
df_wether_final = df_wether_final.drop(['광역시','시군구',], axis=1)
df_wether_final.tail()

,ymd,평균기온,습도,평균풍속,강수량
2192,20160101,1.2,73.0,1.6,0.0
2193,20160102,5.7,77.0,2.0,0.0
2194,20160103,6.5,81.0,1.8,0.0
2195,20160104,2.0,54.0,3.1,0.0
2196,20160105,-2.7,39.0,2.3,0.0


In [144]:
df_wether_final = df_wether_final.fillna(0)
df_wether_final.isnull().sum()

ymd     0
평균기온    0
습도      0
평균풍속    0
강수량     0
dtype: int64

In [150]:
#gs 데이터와 기상 데이터 결합하기

df = pd.merge( df_wether_final, df_GS25_final, on="ymd", how="left")
df.head(20)


,ymd,평균기온,습도,평균풍속,강수량,상품,수량
0,20160101,1.2,73.0,1.6,0.0,라면,7
1,20160101,1.2,73.0,1.6,0.0,과자,655
2,20160101,1.2,73.0,1.6,0.0,라면,282
3,20160101,1.2,73.0,1.6,0.0,마스크,35
4,20160101,1.2,73.0,1.6,0.0,맥주,183
5,20160101,1.2,73.0,1.6,0.0,생리대,78
6,20160101,1.2,73.0,1.6,0.0,생수,56
7,20160101,1.2,73.0,1.6,0.0,숙취해소제,14
8,20160101,1.2,73.0,1.6,0.0,스타킹,28
9,20160101,1.2,73.0,1.6,0.0,아이스크림,211
